In [17]:
from keras.models import Sequential, load_model 
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import preprocessing
import csv
import numpy as np
import pandas as pd

In [2]:
with open('013_b_1114_1_2_p1.csv', newline='') as csvfile1:
    data1 = pd.read_csv(csvfile1)
    data1 = data1.rename(columns = {'p1Cash':'Cash','p1Stock':'Stock','p1TotalAsset':'TotalAsset',
                                   'p1Decision':'Decision','p1ChechHistory':'CheckHistory'})
with open('014_b_1114_1_2_p2.csv', newline='') as csvfile2:
    data2 = pd.read_csv(csvfile2)
    data2 = data2.rename(columns = {'p2Cash':'Cash','p2Stock':'Stock','p2TotalAsset':'TotalAsset',
                                   'p2Decision':'Decision','p2ChechHistory':'CheckHistory'})

In [3]:
X = data1.loc[:,['Cash','StockPrice','TotalAsset']]
X['opp_TotalAsset'] = data2.loc[:,['TotalAsset']]
X['CheckHistory'] = pd.get_dummies(data1.loc[:,'CheckHistory'])['yes']

In [4]:
X_1_101 = X[1:101]
X_1_101 = X_1_101.reset_index()
X_0_100 = X[0:100]
X_0_100 = X_0_100.rename(columns = {'Cash':'pre_Cash','StockPrice':'pre_StockPrice','TotalAsset':'pre_TotalAsset',
                                   'opp_TotalAsset':'pre_opp_TotalAsset','CheckHistory':'pre_CheckHistory'}, inplace = False)
X = pd.concat([X_1_101,X_0_100], axis=1)
X = X.drop(['index'], axis = 1)

In [5]:
X['delta_Price'] = X['StockPrice']-X['pre_StockPrice']
X_data = X[0:99]
X = preprocessing.scale(X_data)

In [6]:
Y_data = data1.loc[:,'Decision']
Y_dum = pd.get_dummies(Y_data)
Y = Y_dum[1:100]

In [23]:
#固定每次random跑出的結果以利解釋
np.random.seed(408)
# model = Sequential()
# model.add(Dense(80, input_dim = 11, activation='relu'))
# model.add(Dense(3, activation='sigmoid'))
# model.summary()

# model.compile(loss='categorical_crossentropy', 
#               optimizer='adam', 
#               metrics=['accuracy'])

# test = model.fit(X,Y, epochs = 1000, batch_size = 80, validation_split = 0.2, shuffle = True)
model = Sequential()
model.add(LSTM(units = 50, input_shape = (11,)))


# model = buildManyToManyModel(X.shape)
callback = EarlyStopping(monitor="accuracy", patience=10, verbose=1, mode="auto")
model.fit(X,Y, epochs = 1000, batch_size = 80, validation_split = 0.2, shuffle = True, callbacks=[callback])


D:\Software\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Software\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(99, 11))`
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 99, 10)            880       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 99, 1)             11        
Total params: 891
Trainable params: 891
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking input: expected lstm_5_input to have 3 dimensions, but got array with shape (99, 11)